In [40]:
from citipy import citipy
from random import uniform
import pandas as pd
import numpy as np
import csv
import random
import matplotlib.pyplot as plt
import requests
import json
import seaborn as sns
import datetime

from config import maps_key, place_key




In [41]:
##generate random lat, long

city = {}
cities = []
def newpoint():
    return uniform(-180,180), uniform(-90, 90)

while len(cities) < 500:
    points = []
    points = list((newpoint() for x in range(500)))
    for i in range(len(points)):
        lat = points[i][0]
        long = points[i][1]
        city[citipy.nearest_city(lat,long).city_name] = points[i]
       # country.append(citipy.nearest_city(lat,long).country_code)
    cities = list(set(city))
    

2500

In [60]:
a = list(city.keys())

city_df = pd.DataFrame([city for city in a], columns=["City"])

# city_df = pd.DataFrame.from_dict(city, orient='columns', dtype=None)
# city_df


city_df["Lat"] = ""
city_df["Long"] = ""


for index, row in city_df.iterrows():
    lat = city[a[index]][0]
    lat = city[a[index]][1]
    city_df.set_value(index, "Lat", lat)
    city_df.set_value(index, "Long", long)
 #   city_df.set_value(index, "Country", country)

In [61]:
city_df

,City,Lat,Long
0,dikson,84.8783,79.7626
1,thinadhoo,66.2808,79.7626
2,ushuaia,-69.1353,79.7626
3,bambous virieux,73.2343,79.7626
4,barentsburg,-9.38855,79.7626
5,yarada,86.3685,79.7626
6,bhitarwar,78.1837,79.7626
7,port elizabeth,29.9953,79.7626
8,ribeira grande,-31.3679,79.7626
9,qaanaaq,-83.2695,79.7626


In [64]:

city_df["Country"] = ""
for index, row in city_df.iterrows():
    target_city = city_df["City"][index]
    target_url = "https://maps.googleapis.com/maps/api/geocode/json?" \
    "address=%s&key=%s" % (target_city, maps_key)
    geo_data = requests.get(target_url).json()
    try:
        country = geo_data["results"][0]["address_components"][3]["long_name"]
    except IndexError:
        try:
            country = geo_data["results"][0]["address_components"][2]["long_name"]
        except IndexError:
            try:
                country = geo_data["results"][0]["address_components"][1]["long_name"]
            except IndexError:
                country = geo_data["results"][0]["address_components"][0]["long_name"]
    print("settings values " +city_df["City"][index])
    city_df.set_value(index, "Country", country)

KeyboardInterrupt: 